# Der gedämpfte harmonische Oszillator
## (symbolische Lösung von DGL mit sympy)

### Das Modell
<center><img src="figs/oszillator_sketch.png" width=400 /></center>

Die Bewegungsgleichung:
\begin{eqnarray}
m\ddot{x}(t) & = & F_K + F_R \\
          & = & -kx(t) - 2\delta\dot{x}(t) \\
\end{eqnarray}

Mit $k=m\omega_0^2$ und $\delta = m\gamma\omega_0$ ergibt sich für die Bewegungsgleichung:

$$
\boxed{\ddot{x}(t) + 2\gamma\omega_0 \dot{x}(t) + \omega_0^2 x(t) = 0}.
$$

Anfangsbedingungen: $x(0)=1; \dot{x}(0)=0$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp

sp.init_printing()

#### Schritt 1: Definiere nötige `sympy`-Größen

In [ ]:
## SOLUTION

t, gamma, omega0 = sp.symbols('t gamma omega0')
x = sp.symbols('x', cls=sp.Function)
x = x(t)

#### Schritt 2: Definiere die zu lösende Differentialgleichung und lasse sie von `sympy` lösen

In [ ]:
## SOLUTION

dgl = sp.Eq(x.diff(t, t) + 2 * omega0 * gamma * x.diff(t) + omega0**2 * x, 0)
dgl

In [ ]:
## SOLUTION

# Es ist nützlich zur Weiterverarbeitung sofort die rechte Seite der
# Ergebnisgleichung zu isolieren:
sol = sp.dsolve(dgl).rhs
sol

#### Schritt 3: Definiere die Anfamgsbedingungen und löse die Differentialgleichung erneut

Unsere Anfangsbedingungen: $\quad x(0) = 1 \quad\text{und}\quad \left.\frac{dx(t)}{dt}\right|_{t=0}=0$

In [ ]:
## SOLUTION

# Die Anfamgswerte sind in einen Dictionary zu definieren:
ics = {x.subs(t, 0) : 1, x.diff(t).subs(t, 0) : 0}
ics

In [ ]:
## SOLUTION

sol = sp.dsolve(dgl, ics=ics).rhs
sp.print_latex(sol)

Wir wissen aus Physik 1 (Mechanik):
1. $\gamma=0$: ungedämpfte harmonische Schwingungen
2. $\gamma > 1$: Keine Schwingungen des Systens; exponentieller Abfall der Anfangsauslenkung 
3. $\gamma < 1$: Schwingungen, deren Amplitude exponentiell kleiner wird
4. $\gamma=1$: Grenzfall zwischen den Fällen 2 und 3

#### Schritt 4: Weiterverarbeitung der Lösung (Testen der einzelnen Fälle)

#### Schritt 4.1: Der Fall $\gamma=0$

Beachte: $\quad\cos(x)=\frac 12(\exp(\rm{i}x)+\exp(-\rm{i}x)) \quad\text{und}\quad\sin(x)=\frac 1{2\rm{i}}(\exp(\rm{i}x)-\exp(-\rm{i}x))$

In [ ]:
## SOLUTION

# Wir müssen die komplexen Exponentialfunktionen in eine
# reelle sinus-Funtion umwandeln
sol_gamma_0 = sol.subs({gamma: 0})
sol_gamma_0.rewrite(sp.cos)

#### Schritt 4.2: Der Fall $\gamma > 1$

In [ ]:
## SOLUTION

# Durch Umschreiben des Ergebnisses sieht man besser, dass die Lösung 
# einen exponentiellen Abfall beschreibt. 
sol_gamma_2 = sol.subs({gamma: 2})
sol_gamma_2

#### Schritt 4.3: Der Fall $\gamma < 1$

In [ ]:
## SOLUTION

# Hier erfordert das Umformen in einen exponentiell abfallende Sinusschwingung
# etwas 'try and error'!
sol_gamma_0p5 = sol.subs({gamma: sp.Rational(1,2)})
sol_gamma_0p5 = sol_gamma_0p5.expand().rewrite(sp.cos).factor().simplify()
sol_gamma_0p5

#### Schritt 4.4: Der Fall $\gamma = 1$

In [ ]:
## SOLUTION

# Durch umformen der Lösung sieht man, dass für gamma=1 sowohl Zähler als auch
# Nenner gegen 0 gehen.
sol_fac = sol.factor()
sol_fac

In [ ]:
## SOLUTION

# Hier ein sympy Weg, um zu sehen, dass Zähler und Nenner der allgemeinen
# Lösung für gamma=1 formal zu Null werden.
z, n = sp.fraction(sol_fac)
z.subs({gamma: 1}), n.subs({gamma: 1})

Da Zähler und Nenner der Lösung für $\gamma=1$ formal zu Null werden,
ist hier eine Limes-Betrachtung der Lösung erforderlich:

$$
\lim_{\gamma\to 1}  \frac{\left(\gamma e^{2 \omega_{0} t \sqrt{\gamma^{2} - 1}} - \gamma + \sqrt{\gamma^{2} - 1} e^{2 \omega_{0} t \sqrt{\gamma^{2} - 1}} + \sqrt{\gamma^{2} - 1}\right) e^{- \gamma \omega_{0} t} e^{- \omega_{0} t \sqrt{\gamma^{2} - 1}}}{2 \sqrt{\left(\gamma - 1\right) \left(\gamma + 1\right)}}\quad = \quad ?
$$

In [ ]:
## SOLUTION

sol_gamma_1 = sol_fac.limit(gamma, 1)
sol_gamma_1

#### Schritt 5: Plotten der Ergebnisse

`sympy`-Ausdrücke können in `numpy`-Funktionen umgewandelt werden.  Dies erlaubt z.B. eine Weiterverarbeitung mit `scipy` oder `matplotlib`. 

In [ ]:
## SOLUTION

# Zum Plotten wandeln wir einen sympy-Ausdruck in eine numpy-Funtion um:
sol_n = sp.lambdify(t, sol.subs({gamma: 0.1, omega0: sp.pi}))

t_n = np.linspace(0.0, 5.0, 100)
plt.plot(t_n, sol_n(t_n).real)
plt.plot(t_n, np.exp(-1./10. * np.pi * t_n))

In [ ]:
## SOLUTION

# Plot für mehrere Fälle
t_n = np.linspace(0.0, 4, 1000)
for i in (0., 0.2, 0.5, 1.0, 2.0):
    # Da wir für gamma=1 die Lösung bearbeiten müssen, brauchen
    # wir hier eine Fallunterscheidung
    if np.isclose(i, 1):
        sol_n = sp.lambdify(t, sol_gamma_1.subs({omega0: sp.pi}))
    else:
        sol_n = sp.lambdify(t, sol.subs({gamma: i, omega0: sp.pi}))

    plt.plot(t_n, sol_n(t_n).real, label=f'gamma: {i}')
   
plt.xlabel("t")
plt.ylabel("x(t)")
plt.legend()